In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint  
import pandas as pd

# Create an instance of MongoClient
client = MongoClient(port=27017)

# Assign the uk_food database to a variable name
db = client['uk_food']  

# Assign the collection to a variable
establishments = db['establishments']  


# Which establishments have a hygiene score equal to 20?
hygiene_20_count = establishments.count_documents({"scores.Hygiene": 20})
pprint(establishments.find_one({"scores.Hygiene": 20}))
hygiene_20_df = pd.DataFrame(list(establishments.find({"scores.Hygiene": 20})))
print("Number of rows:", len(hygiene_20_df))
print(hygiene_20_df.head(10))

# Which establishments in London have a RatingValue greater than or equal to 4?
london_greater_than_4_count = establishments.count_documents({"LocalAuthorityName": {"$regex": "London", "$options": "i"}, "RatingValue": {"$gte": 4}})
pprint(establishments.find_one({"LocalAuthorityName": {"$regex": "London", "$options": "i"}, "RatingValue": {"$gte": 4}}))
london_greater_than_4_df = pd.DataFrame(list(establishments.find({"LocalAuthorityName": {"$regex": "London", "$options": "i"}, "RatingValue": {"$gte": 4}})))
print("Number of rows:", len(london_greater_than_4_df))
print(london_greater_than_4_df.head(10))

# What are the top 5 establishments with a RatingValue of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?
penang_flavours_location = establishments.find_one({"BusinessName": "Penang Flavours"})["geocode"]
nearest_establishments = establishments.find({"RatingValue": 5, "geocode.latitude": {"$gte": penang_flavours_location["latitude"] - 0.01, "$lte": penang_flavours_location["latitude"] + 0.01}, "geocode.longitude": {"$gte": penang_flavours_location["longitude"] - 0.01, "$lte": penang_flavours_location["longitude"] + 0.01}}).sort([("scores.Hygiene", 1)]).limit(5)
for establishment in nearest_establishments:
    pprint(establishment)
nearest_establishments_df = pd.DataFrame(list(nearest_establishments))
print("Number of rows:", len(nearest_establishments_df))
print(nearest_establishments_df)

# How many establishments in each Local Authority area have a hygiene score of 0? Sort the results from highest to lowest, and print out the top ten local authority areas.
zero_hygiene_count_by_local_authority = establishments.aggregate([
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 10}
])
for item in zero_hygiene_count_by_local_authority:
    pprint(item)



None
Number of rows: 0
Empty DataFrame
Columns: []
Index: []
None
Number of rows: 0
Empty DataFrame
Columns: []
Index: []


TypeError: 'NoneType' object is not subscriptable